## ITERATION

In [51]:
for element in [1, 2, 3]:
    print(element)
for element in (1, 2, 3):
    print(element)
for key in {'one':1, 'two':2}:
    print(key)
for char in "123":
    print(char)
for line in open("myfile.txt"):
    print(line, end='')

1
2
3
1
2
3
one
two
1
2
3


FileNotFoundError: [Errno 2] No such file or directory: 'myfile.txt'

Behind the scenes.
- forloop은 container object에 iter()함수를 호출합니다.
- return받은 객체의 __next__() 를 호출하면서 iterating할 대상을 하나씩 가져옵니다.
- StopIteration 오류가 발생하면 forloop을 멈추게 됩니다.

```
>>> s = 'abc'
>>> it = iter(s)
>>> it
<iterator object at 0x00A1DB50>
>>> next(it)
'a'
>>> next(it)
'b'
>>> next(it)
'c'
>>> next(it)
Traceback (most recent call last):
  File "<stdin>", line 1, in ?
    next(it)
StopIteration
```

In [54]:
# Iteration protocol을 구현해보기
class MyClass:
    def __init__(self):
        self._data = [4,0,7,3]
        self.index = 0
    def __iter__(self):
        return self
    def __next__(self):
        if len(self._data) == self.index:
            raise StopIteration
        ret = self._data[self.index]
        self.index += 1
        return ret

for x in MyClass():
    print(x)

4
0
7
3


In [55]:
from collections.abc import Iterable
isinstance(MyClass(), Iterable)

True

## GENERATOR (PEP 255)

- iterator를 return하는 함수입니다.
- 일반 함수처럼 생겼는데 다만 함수 내 **yield** keyword가 사용됩니다.
- (신기하게도) 함수가 **동작중 SUSPEND**될 수 있습니다.
- 물론 SUSPEND되었던 부분에서 이어 다시 수행할 수 있습니다.

In [36]:
def yield_test():
    return 10
yield_test()

10

In [38]:
# yield가 사용되는 순간 함수가 아닌 generator로 변신.
def yield_test():
    # return 10
    yield 10
yield_test()

<generator object yield_test at 0x0000000006BEC9E8>

In [39]:
# iterator를 return하기 때문에 looping가능.
for x in yield_test():
    print(x)

10


In [44]:
# 일반함수와 유사합니다.
def yield_test():
    yield 1
    yield 2
    yield 3

for x in yield_test():
    print(x)

1
2
3


In [47]:
# iterator를 return하는
g = yield_test()
print('__iter__' in dir(g))
print('__next__' in dir(g))
next(g), next(g), next(g)

True
True


(1, 2, 3)

In [42]:
# 함수 수행중 중단 및 재개
def yield_test():
    yield 1
    print('resumed')
    yield 2
    print('resumed')
    yield 3
for x in yield_test():
    print(x)

1
resumed
2
resumed
3


In [3]:
import random

def lottery():
    for i in range(6):
        yield random.randint(1,40)
        yield random.randint(1,15)

for random_number in lottery():
    print("And the next number is... %d!" %(random_number))

And the next number is... 18!
And the next number is... 9!
And the next number is... 35!
And the next number is... 1!
And the next number is... 14!
And the next number is... 13!
And the next number is... 31!
And the next number is... 2!
And the next number is... 16!
And the next number is... 13!
And the next number is... 12!
And the next number is... 3!


In [4]:
# fill in this function
def fib():
    pass #this is a null statement which does nothing when executed, useful as a placeholder.

# testing code
import types
if type(fib()) == types.GeneratorType:
    print("Good, The fib function is a generator.")

    counter = 0
    for n in fib():
        print(n)
        counter += 1
        if counter == 10:
            break

In [57]:
def fib():
    val1, val2 = 1, 1
    while True:
        yield val1
        val1, val2 = val2, (val1+val2)

# testing code
import types
if type(fib()) == types.GeneratorType:
    print("Good, The fib function is a generator.")

    counter = 0
    for n in fib():
        print(n)
        counter += 1
        if counter == 10:
            break

Good, The fib function is a generator.
1
1
2
3
5
8
13
21
34
55


LIST와 무엇이 다른가요?
- 한번 소비되면 끝이에요.
- 데이터를 무한으로 생성 가능해요.
- LIST보다 메모리를 조금 써요.
- LIST사용하는 것보다는 조금 느릴 수 있어요.
- len이나 slice를 generator에는 사용할 수 없습니다.

return을 만나거나 함수끝까지 가면 generator는 끝납니다.

python3의 많은 함수들이 generator를 사용합니다. python2에서는 그렇지 않았었는데!
 - laziness가 python3의 주요 테마이자 trend입니다.

```
# python2
range(10), xrange(10)
open(file).readlines, open(file).xreadlines
{}.keys()

# python3
range(10)
open(file).readlines()
{}.keys()

# 엄밀히 구현내용을 보면 xrange는 generator처럼 동작할뿐, generator자체는 아니라고 합니다.
```

기본적으로 list를 return하는 함수를 generator로 바꿔서 사용할 수 있습니다.
- 한번 쓰고 재사용되지 않을 것이라면
- 메모리를 과점유하고 싶지 않다면
- database record fetching

In [49]:
def counter_list(size):
    results = []
    cur = 1
    while cur <= size:
        results.append(cur)
        cur = cur + 1
    return results

def counter_gen(size):
    cur = 1
    while cur <= size:
        yield cur
        cur = cur + 1

list(counter_gen(500)) == counter_list(500)

True

## GENERATOR EXPRESSION (AGAIN)

In [8]:
[str(num) for num in range(3)]

['0', '1', '2']

In [10]:
(str(num) for num in range(3))

<generator object <genexpr> at 0x00000000068E65C8>

In [12]:
# generator는 한번 사용하면 버려져요!
g = (str(num) for num in range(3))
list(g), list(g)

(['0', '1', '2'], [])

In [59]:
# 함수의 단일인자로 사용할때는 ()가 필요없어집니다.
sum((x+2 for x in range(10))), \
sum(x+2 for x in range(10))

(65, 65)

연습문제)
list를 인자로 받아 음수를 filtering하는 generator를 만들어보세요.

```
>> list(pos_gen(range(-10,10)))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
```

In [21]:
def pos_gen(seq):
    for x in seq:
        if x >= 0:
            yield x
list(pos_gen(range(-10,10)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [23]:
def pos_gen(seq):
    return (x for x in seq if x >= 0)
list(pos_gen(range(-10,10)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [33]:
pos_gen = lambda seq: filter(lambda x: x>=0, seq)
list(pos_gen(range(-10,10)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

(Exercise)영어문장을 받아서 각 단어의 앞글자만 대문자로 유지해주는 함수를 만들어봅시다. (문자열의 capitalize method를 사용하세요.)

```
>> capwords(" abc  dEf ")
"Abc Def"
```

In [56]:
# Lib/string.py 표준라이브러리에서 
# generator expression 사용된 것이 보이시나요?

# Capitalize the words in a string, e.g. " aBc  dEf " -> "Abc Def".
def capwords(s, sep=None):
    """capwords(s [,sep]) -> string

    Split the argument into words using split, capitalize each
    word using capitalize, and join the capitalized words using
    join.  If the optional second argument sep is absent or None,
    runs of whitespace characters are replaced by a single space
    and leading and trailing whitespace are removed, otherwise
    sep is used to split and join the words.

    """
    return (sep or ' ').join(x.capitalize() for x in s.split(sep))

(Exercise) 1천억건의 데이터를 가진 list를 생성해주세요. 그 중에 앞 두개만 출력하겠습니다.

In [1]:
range(1_000_000_000_000)

range(0, 1000000000000)

In [61]:
r = range(1_000_000_000_000)
i = iter(r)
next(i)
next(i)

1

내가 요구할때 하나씩 주시면 됩니다.

- 메모리를 최대한 적게 사용하고
- 가능한 처리를 **늦게까지** 미루는

PYTHON에서 사용하는 거의 모든 COLLECTION은 iterable합니다. 일관된 ~~사용자경험~~**개발자 경험**을 제공합니다.

## COROUTINE (PEP 492) with asyncio (PEP3153)

PYTHON3에서 제공하는 ASYNCIO를 이용해서 COROUTINE을 쉽게 생성할 수 있습니다.

In [6]:
import asyncio

In [7]:
async def greeting():
    print("Hello World")

greeting()

<coroutine object greeting at 0x104404780>

In [8]:
loop = asyncio.get_event_loop()
loop.run_until_complete(greeting())
loop.close()

Hello World


In [9]:
async def compute(future):
    print('starting...')
    res = await answer()
    future.set_result(res)
async def answer():
    await asyncio.sleep(1)
    return 42

f = asyncio.Future()
loop = asyncio.get_event_loop()
loop.run_until_complete(compute(f))
loop.close()
f.result()

RuntimeError: Event loop is closed